In [228]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import config
import logging
import importlib
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import validators
import pickle
import time
import random
from calendar import monthrange
import re
import spacy
from spacy.tokens import Doc, Span
import nltk
from nltk.corpus import stopwords
from nltk.corpus import treebank
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import wordnet

from nltk import pos_tag, sent_tokenize, wordpunct_tokenize

pd.options.display.max_columns = 300
import string
import unicodedata

pd.options.display.max_columns = 200
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt  

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as datasets
import pandas as pd
import numpy as np
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import mean_squared_error
from pandas.plotting import scatter_matrix
import seaborn as sns
import scipy.stats as stats
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.svm import SVC  
from time import time
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer


import gensim
from gensim import corpora
from gensim import models
from gensim.sklearn_api import lsimodel, ldamodel
from gensim.matutils import sparse2full
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel

import pyLDAvis
import pyLDAvis.gensim

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from functools import partial
from operator import itemgetter
from joblib import dump, load

## Get our full populate of clean news articles (across three sources:  NYT, FT, Reuters).

In [158]:
with open('full_sampled_pop.pkl', 'rb') as f:
    news_df = pickle.load(f)

In [159]:
len(news_df)

6669

In [162]:
news_df.rename(columns={'bigrammed_text': 'docs'}, inplace=True)

In [220]:
news_df.head()

,paper,docs,text,docs_text
240,NYT,"[allies, president, vladimir, v, russia, suffe...",MOSCOW — Allies of President Vladimir V. Putin...,allies president vladimir v russia suffer sign...
2306,NYT,"[president, warn, tuesday, attack, iran, might...",WASHINGTON — President Trump warned on Tuesday...,president warn tuesday attack iran might carry...
2106,RT,"[german, government, friday, present, far-reac...",FRANKFURT/The German government on Friday pres...,german government friday present far-reaching ...
236,FT,"[many, mexican, business, people, armando, san...","Like many Mexican business people, Armando San...",many mexican business people armando santacruz...
333,FT,"[charles, li, chief, executive, hong kong, exc...","Charles Li, chief executive of Hong Kong Excha...",charles li chief executive hong kong exchanges...


In [221]:
news_doc_pop_df = news_df[['paper', 'text']]

In [224]:
len(news_doc_pop_df)

6669

In [225]:
with open('news_doc_pop_df.pkl', 'wb') as f:
    pickle.dump(news_doc_pop_df, f)

## "Stringify" our doc tokens for SKLearn

In [163]:
news_df['docs_text'] = news_df.docs.apply(lambda x: ' '.join(x))

In [164]:
news_df.head()

,paper,docs,text,docs_text
240,NYT,"[allies, president, vladimir, v, russia, suffe...",MOSCOW — Allies of President Vladimir V. Putin...,allies president vladimir v russia suffer sign...
2306,NYT,"[president, warn, tuesday, attack, iran, might...",WASHINGTON — President Trump warned on Tuesday...,president warn tuesday attack iran might carry...
2106,RT,"[german, government, friday, present, far-reac...",FRANKFURT/The German government on Friday pres...,german government friday present far-reaching ...
236,FT,"[many, mexican, business, people, armando, san...","Like many Mexican business people, Armando San...",many mexican business people armando santacruz...
333,FT,"[charles, li, chief, executive, hong kong, exc...","Charles Li, chief executive of Hong Kong Excha...",charles li chief executive hong kong exchanges...


## Pull out our features and labels and run train test split

In [167]:
X_all = news_df[['docs_text','text']]

In [168]:
X_all.head()

,docs_text,text
240,allies president vladimir v russia suffer sign...,MOSCOW — Allies of President Vladimir V. Putin...
2306,president warn tuesday attack iran might carry...,WASHINGTON — President Trump warned on Tuesday...
2106,german government friday present far-reaching ...,FRANKFURT/The German government on Friday pres...
236,many mexican business people armando santacruz...,"Like many Mexican business people, Armando San..."
333,charles li chief executive hong kong exchanges...,"Charles Li, chief executive of Hong Kong Excha..."


In [169]:
y_all = news_df[['paper']]

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 42)

In [171]:
X_train.head()

,docs_text,text
2158,priests robe hard hat worship saturday cathedr...,PARIS — Priests in robes and hard hats worship...
1722,death toll earthquake pakistan tuesday risen m...,The death toll from an earthquake in Pakistan ...
1009,military court algeria sentence bouteflika you...,A military court in Algeria has sentenced Said...
657,united states endorse turkey plan offensive no...,The United States does not endorse Turkey’s pl...
1697,reporters without borders advocate press freed...,"LONDON — Reporters Without Borders, which advo..."


## Get a TF-IDF Vector and try to model with that (first)

In [172]:
tfidf = TfidfVectorizer(stop_words='english', min_df=5, max_df=.5)

In [173]:
X_train_tfidf_vec = tfidf.fit_transform(X_train.docs_text)

## Persist our TF-IDF Vector Transformer

In [229]:
dump(tfidf, 'tfidf.joblib')

['tfidf.joblib']

## Now that our TF-IDF vector is fitted on our training data, transform our test data

In [174]:
X_test_idf_vec = tfidf.transform(X_test.docs_text)

## Get our TF-IDF word frequency counts

In [175]:
sum_words = X_train_tfidf_vec.sum(axis=0) 

In [176]:
tfidf_word_metrics = [(word, sum_words[0, idx]) for word, idx in     tfidf.vocabulary_.items()]

In [177]:
sorted_tfidf_word_metrics = sorted(tfidf_word_metrics, key = lambda x: x[1], reverse=True)

In [178]:
sorted_tfidf_word_metrics

[('brexit', 115.47312406925798),
 ('iran', 115.28576357020368),
 ('china', 109.43335202006621),
 ('president', 106.47342592854922),
 ('party', 104.91487101161685),
 ('police', 99.99621373554963),
 ('deal', 99.24001215901592),
 ('united', 97.85395615031742),
 ('minister', 97.53146350410645),
 ('hong', 96.3545287533691),
 ('kong', 94.60109447568769),
 ('official', 83.68999056703814),
 ('attack', 83.35716310490925),
 ('delay', 82.4146289417054),
 ('state', 80.20101584449804),
 ('exchange', 80.16639384138414),
 ('quote', 80.05848822911842),
 ('reporting', 79.38230027253651),
 ('list', 79.36187700678323),
 ('election', 78.97162398594911),
 ('complete', 78.76627843208652),
 ('saudi', 78.4886147555002),
 ('european', 78.47458807606736),
 ('minute', 78.3395270512687),
 ('minimum', 78.04444682690901),
 ('delayed', 77.78295619857415),
 ('states', 76.00859719793846),
 ('prime', 74.6580463925685),
 ('week', 70.24668965298628),
 ('cent', 68.78395166748244),
 ('protest', 68.76518199537566),
 ('leade

In [29]:
X_train_tfidf_vec.shape

(5335, 16436)

## Helper function to print model metrics

In [36]:
def print_metrics(test, preds):
    print("Precision Score: {}".format(precision_score(test, preds, average='macro')))
    print("Recall Score: {}".format(recall_score(test, preds, average='macro')))
    print("Accuracy Score: {}".format(accuracy_score(test, preds)))
    print("F1 Score: {}".format(f1_score(test, preds, average='macro')))

## First do a "dummy" classifier to obtain a baseline

In [179]:
dummy = DummyClassifier()
dummy.fit(X_train_tfidf_vec.toarray(), y_train)

DummyClassifier(constant=None, random_state=None, strategy='stratified')

In [180]:
dum_pred = dummy.predict(X_test_idf_vec)

In [181]:
print_metrics(y_test, dum_pred)

Precision Score: 0.33408081199657874
Recall Score: 0.33405137209427865
Accuracy Score: 0.3410794602698651
F1 Score: 0.3340491343429209


In [182]:
model = GaussianNB()

# Train the model using the training sets
model.fit(X_train_tfidf_vec.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

## Persist our GaussianNB Model

In [230]:
dump(model, 'gnb-model.joblib')

['gnb-model.joblib']

In [183]:
train_pred = model.predict(X_train_tfidf_vec.toarray())

In [184]:
print_metrics(y_train, train_pred)

Precision Score: 0.9938240192291183
Recall Score: 0.9932700006968456
Accuracy Score: 0.9932521087160262
F1 Score: 0.9935071093941902


In [185]:
pred = model.predict(X_test_idf_vec.toarray())

In [186]:
print_metrics(y_test, pred)

Precision Score: 0.9088353952584769
Recall Score: 0.9114490589625236
Accuracy Score: 0.9077961019490255
F1 Score: 0.9082004922623916


In [51]:
ft_text = '''A Russian cyber espionage unit has hacked Iranian hackers to lead attacks in more than 35 countries, a joint UK and US investigation has revealed.

The so-called Turla group, which has been linked with Russian intelligence, allegedly hijacked the tools of Oilrig, a group widely linked to the Iranian government, according to a two-year probe by the UK’s National Cyber Security Centre in collaboration with the US’ National Security Agency. The NCSC is part of GCHQ, the digital intelligence agency.

The Iranian group is most likely unaware that its hacking methods have been hacked and deployed by another cyber espionage team, security officials involved in the investigation said. Victims include military establishments, government departments, scientific organisations and universities across the world, mainly in the Middle East.

Paul Chichester, NCSC director of operations, said Turla’s activity represented “a real change in the modus operandi of cyber actors” which he said “added to the sense of confusion” over which state-backed cyber groups had been responsible for successful attacks.

“The reason we are [publicising] this is because of the different tradecraft we are seeing Turla use,” he told reporters. “We want others to be able to understand this activity.”

Mr Chichester described how Turla began “piggybacking” on Oilrig’s attacks by monitoring an Iranian hack closely enough to use the same backdoor route into an organisation or to gain access to the resulting intelligence. Turla is also known as Waterbug or Venomous Bear.

But the Russian group then progressed to initiating their own attacks using Oilrig’s command-and-control infrastructure and software. Organisations in approximately 20 countries were successfully hacked in this way.

“[Turla] could benefit from the operations of Oilrig. They could collect some of their operational output . . . It allowed them to gain more rapid access to victims than they would otherwise have done,” Mr Chichester said. “It made life much easier. This is an opportunistic operation which has given [Turla] a wealth of information and access they wouldn’t otherwise have had.”

The Kremlin did not respond to a request for comment from the Financial Times. Russia’s government has consistently denied it is behind hacking attempts on other states. President Vladimir Putin, in an interview with the FT earlier this year, described allegations that Moscow had orchestrated attempts to influence the 2016 US elections as “mythical”.

Cyber espionage groups are increasingly concealing their identities under so called “false flag” operations — in which they try to mimic the activities of another group. Last year US intelligence agencies were reported to have uncovered the fact that Russian hackers had attempted to disrupt the Winter Olympics in Pyeongchang, South Korea, using lines of code associated with Lazarus Group, attributed to North Korea.

But NCSC says Turla’s operations go far further than imitation, and that Oilrig itself — also known by the names Crambus and APT34 — was hacked.

“We have never seen this done to the level of sophistication that we are seeing here,” Mr Chichester said. “It’s unique in the complexity and scale and sophistication. It’s actually really hard masquerading [as another entity].”

He said that Turla now had the potential to hijack other state-sponsored cyber groups. “This is becoming a very crowded space and we do see people innovate quite rapidly in that domain,” he said.'''

In [52]:
ft_text

'A Russian cyber espionage unit has hacked Iranian hackers to lead attacks in more than 35 countries, a joint UK and US investigation has revealed.\n\nThe so-called Turla group, which has been linked with Russian intelligence, allegedly hijacked the tools of Oilrig, a group widely linked to the Iranian government, according to a two-year probe by the UK’s National Cyber Security Centre in collaboration with the US’ National Security Agency. The NCSC is part of GCHQ, the digital intelligence agency.\n\nThe Iranian group is most likely unaware that its hacking methods have been hacked and deployed by another cyber espionage team, security officials involved in the investigation said. Victims include military establishments, government departments, scientific organisations and universities across the world, mainly in the Middle East.\n\nPaul Chichester, NCSC director of operations, said Turla’s activity represented “a real change in the modus operandi of cyber actors” which he said “added

In [226]:
with open('test_article.pkl', 'wb') as f:
    pickle.dump(ft_text, f)

## Gotta move this class and this function to a module!

In [53]:

class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, language='english', source=None):
        self.source = source
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.more_stops = {'mr', 'ms', 'mr.', 'ms.', 'mrs', 'mrs.','say', 'said', 'saying', 
                           'also', 'yeh', 'hom', 'even', 'like', 'k', 'n', 'u', 'would', 'could', '$'}
        self.stopwords.update(self.more_stops) 
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(unicodedata.category(char).startswith('P') for char in token)

    def is_stopword(self, token):
        return token.lower() in self.stopwords
    
    def get_stopwords(self):
        return self.stopwords

    def get_wordnet_pos(selfm, word):
        #Map POS tag to first character lemmatize() 
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    def normalize(self, document):
        NYT_PAT = '^[A-Z]* — '
        NYT_COMP_PAT = re.compile(NYT_PAT)        
        
        norm_toks = []
    
        if self.source and self.source == 'NYT':
            document = NYT_COMP_PAT.sub('', document)
# I found nltk's sentenizer to perform poorly
#        for sent in sent_tokenize(document):
        for sent in document.split('.'):            
            for tok in nltk.word_tokenize(sent):
                if self.is_punct(tok) or tok.isdigit():
                    continue
                lem = self.lemmatize(tok, self.get_wordnet_pos(tok)).lower()
                if not self.is_stopword(lem):
                    norm_toks.append(lem) 
                
        return norm_toks


    def lemmatize(self, token, pos):
        return self.lemmatizer.lemmatize(token, pos)

    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        for doc in documents:
            yield self.normalize(document)

In [54]:
# code to pull out our custom bi-grams
bigrams = ['donald trump', 'angela merkel', 'boris johnson', 'vladimir putin', 'benjamin netanyahu', 
           'hong kong', 'north korea', 'south korea', 'united kingdom', 'united states', 'south africa', 
           'xi jinping', 'carrie lam', 'oleksiy honcharuk', 'volodymyr zelensky', 'emmanuel macron',
           'viktor orbán', 'justin trudeau', 'north america', 'south america', 'sinn fein', 'jeremy corbyn', 'narendra modi',
            'mohamed morsi', 'shuping wang', 'hassan rouhani', 'rudy giuliani', 'joe biden', 'new zealand', 'european union', 'pope francis']

In [55]:
class BiGramTransformer:
    def __init__(self, bigram_list):
        self.bigram_list = bigram_list        
        self.first_grams = [word.split()[0] for word in bigram_list]
        self.second_grams = [word.split()[1] for word in bigram_list]

    def get_custom_bigrams(self, tokens):
        new_toks = []
        length = len(tokens)

        for ix, tok in enumerate(tokens):
            try:
                if tok in self.first_grams:
                    if ix+1 <= length and tokens[ix+1] in self.second_grams:
                        new_toks.append(' '.join([tok, tokens[ix+1]]))
                        continue
                if tok in self.second_grams:
                    # we assume if we see the 2nd token in a bigram that
                    # we've already prcessed the first and second together
                    # so skip 
                    continue
                new_toks.append(tok)
            except IndexError:
                continue

        return new_toks

In [56]:
txtNorm = TextNormalizer()

In [58]:
ft_text_txfm = txtNorm.normalize(ft_text)

In [59]:
bigram_txfm = BiGramTransformer(bigrams)

In [60]:
ft_text_bi = bigram_txfm.get_custom_bigrams(ft_text_txfm)

['russian',
 'cyber',
 'espionage',
 'unit',
 'hack',
 'iranian',
 'hacker',
 'lead',
 'attack',
 'country',
 'joint',
 'uk',
 'us',
 'investigation',
 'reveal',
 'so-called',
 'turla',
 'group',
 'link',
 'russian',
 'intelligence',
 'allegedly',
 'hijack',
 'tool',
 'oilrig',
 'group',
 'widely',
 'link',
 'iranian',
 'government',
 'accord',
 'two-year',
 'probe',
 'uk',
 'national',
 'cyber',
 'security',
 'centre',
 'collaboration',
 'us',
 'national',
 'security',
 'agency',
 'ncsc',
 'part',
 'gchq',
 'digital',
 'intelligence',
 'agency',
 'iranian',
 'group',
 'likely',
 'unaware',
 'hack',
 'method',
 'hack',
 'deployed',
 'another',
 'cyber',
 'espionage',
 'team',
 'security',
 'official',
 'involve',
 'investigation',
 'victims',
 'include',
 'military',
 'establishment',
 'government',
 'department',
 'scientific',
 'organisation',
 'university',
 'across',
 'world',
 'mainly',
 'middle',
 'east',
 'paul',
 'chichester',
 'ncsc',
 'director',
 'operation',
 'turla',
 'act

In [62]:
ft_text_full_str = ' '.join(ft_text_bi)

In [66]:
ft_series = pd.Series(ft_text_full_str)

In [67]:
ft_test_label = pd.Series('FT')

In [68]:
ft_test_vec = tfidf.transform(ft_series)

In [69]:
ft_text_pred = model.predict(ft_test_vec.toarray())

In [70]:
print_metrics(ft_test_label, ft_text_pred)

Precision Score: 0.0
Recall Score: 0.0
Accuracy Score: 0.0
F1 Score: 0.0


In [71]:
ft_text_pred

array(['NYT'], dtype='<U3')

## Do some analysis using confusion matrix

In [77]:
 confusion_matrix(y_test, pred, labels=["NYT", "FT", "RT"])

array([[447,  14,   9],
       [ 17, 329,   7],
       [ 51,  25, 435]])

In [187]:
test_df = pd.DataFrame(y_test)

In [188]:
pred_df = pd.DataFrame(pred)

In [189]:
pred_df.reset_index(drop=True, inplace=True)

In [190]:
test_df.reset_index(drop=True, inplace=True)

In [191]:
X_test_copy = X_test.copy(deep=True)

In [192]:
X_test_copy.reset_index(drop=True, inplace=True)

In [193]:
X_test_copy.head()

,docs_text,text
0,british prime minister boris johnson sunday re...,"MANCHESTER, EnglandBritish Prime Minister Bori..."
1,seoul south korea north korea sunday two proje...,"SEOUL, South Korea — North Korea said on Sunda..."
2,pakistan ambassador united states raise possib...,Pakistan’s ambassador to the United States rai...
3,romford england regardless whether british vot...,"ROMFORD, England — Regardless of whether Briti..."
4,iran revolutionary guards seize vessel gulf al...,Iran’s Revolutionary Guards have seized a vess...


In [194]:
test_comp_df = pd.concat([pred_df,test_df,X_test_copy], ignore_index=True, axis=1)

In [197]:
test_comp_df.head()

,Predicted,Test,Features,3
0,RT,RT,british prime minister boris johnson sunday re...,"MANCHESTER, EnglandBritish Prime Minister Bori..."
1,NYT,NYT,seoul south korea north korea sunday two proje...,"SEOUL, South Korea — North Korea said on Sunda..."
2,NYT,NYT,pakistan ambassador united states raise possib...,Pakistan’s ambassador to the United States rai...
3,FT,NYT,romford england regardless whether british vot...,"ROMFORD, England — Regardless of whether Briti..."
4,RT,RT,iran revolutionary guards seize vessel gulf al...,Iran’s Revolutionary Guards have seized a vess...


In [201]:
test_comp_df.rename(columns={0:'Predicted', 1:'Test', 2:'Features', 3:'Text'}, inplace=True)

In [211]:
test_comp_df.head(100)

,Predicted,Test,Features,Text
0,RT,RT,british prime minister boris johnson sunday re...,"MANCHESTER, EnglandBritish Prime Minister Bori..."
1,NYT,NYT,seoul south korea north korea sunday two proje...,"SEOUL, South Korea — North Korea said on Sunda..."
2,NYT,NYT,pakistan ambassador united states raise possib...,Pakistan’s ambassador to the United States rai...
3,FT,NYT,romford england regardless whether british vot...,"ROMFORD, England — Regardless of whether Briti..."
4,RT,RT,iran revolutionary guards seize vessel gulf al...,Iran’s Revolutionary Guards have seized a vess...
5,RT,RT,-backed syrian democratic forces saturday hesi...,The U.S.-backed Syrian Democratic Forces said ...
6,NYT,NYT,auckland new zealand standoff indigenous group...,"AUCKLAND, New Zealand — A standoff between Ind..."
7,NYT,NYT,ursula von der leyen incoming chief european u...,"BRUSSELS — When Ursula von der Leyen, the inco..."
8,RT,RT,head group eurosceptic lawmaker prime minister...,The head of a group of eurosceptic lawmakers i...
9,RT,FT,administration issue visa iranian president ha...,The Trump administration has issued visas to I...


In [202]:
test_analysis_df = pd.DataFrame(test_comp_df[test_comp_df.Predicted != test_comp_df.Test])

In [203]:
len(test_analysis_df)

123

In [209]:
test_analysis_df.reset_index(drop=True, inplace=True)

In [210]:
test_analysis_df.head(123)

,Predicted,Test,Features,Text
0,FT,NYT,romford england regardless whether british vot...,"ROMFORD, England — Regardless of whether Briti..."
1,RT,FT,administration issue visa iranian president ha...,The Trump administration has issued visas to I...
2,NYT,RT,survivor relative men die nine year ago one ne...,(Reuters) - A survivor and relatives of 29 men...
3,NYT,FT,hardcore pro-democracy protester left trail ha...,Hardcore pro-democracy protesters left a trail...
4,NYT,RT,democratic republic congoa landslide shutter g...,"GOMA, Democratic Republic of CongoA landslide ..."
5,FT,RT,kuwait strategic foodstuff reserve cover four ...,Kuwait has strategic foodstuff reserves to cov...
6,RT,NYT,indonesia constitutional court thursday reject...,BANGKOK — Indonesia’s Constitutional Court on ...
7,FT,RT,wis terry dittrich work donald trump re-electi...,"WAUKESHA, Wis.Terry Dittrich has been working ..."
8,FT,RT,ancient british structure old pyramid threaten...,"ORKNEY, ScotlandAncient British structures old..."
9,FT,NYT,evidence paris police department miss warn sig...,PARIS — Evidence that the Paris police departm...


In [214]:
test_comp_df.iloc[22].Text # NYT - correct prediction - 7482

'HONG KONG — For the second day in a row, thousands rallied on Sunday in Hong Kong to protest mob violence and what they say is police brutality against peaceful marchers. And for the second consecutive day, the police fired tear gas and rubber bullets at the throngs.On Saturday, riot police officers unleashed pepper spray and rubber bullets and clashed briefly with demonstrators in a train station in a northern town. Barely 24 hours later, clashes erupted again that turned parts of downtown Hong Kong into a tear gas-filled battlefield as riot officers beat back demonstrators who tried to approach the Chinese government’s representative office in the territory.The police on Sunday fired dozens of rounds of tear gas starting from about 7 p.m. and charged protesters, sending hundreds of bystanders who were watching nearby scrambling for shelter. Groups of riot officers with clubs chased black-clad protesters down alleys in the dense urban district.In some areas, as the police sought to d

In [216]:
test_analysis_df.iloc[15].Text # Predicted FT - but NYT - 7202

'HONG KONG — Southeast Asian countries tend to be deeply reluctant to collectively challenge China’s growing military and economic prowess in their region. But this week, they appear to be doing just that — by holding their first joint naval drills with the United States Navy.The drills, which will take place partly in the South China Sea, a site of geopolitical tension, began on Monday. They were not expected to focus on lethal maneuvers, or to take place in contested waters where China operates military bases.But the maneuvers follow similar exercises held last year by China and the 10-member Association of Southeast Asian Nations in an undisputed area of the sea, making them a riposte of sorts to Beijing.During a summer of heightened tensions over territorial claims, plus an escalating trade war between China and the United States, the drills are being closely watched as the latest move in a high-stakes geopolitical chess match between the superpowers and their shared regional allie

In [218]:
test_analysis_df.iloc[3].Text # Predicted NYT - but FT - 3557

'Hardcore pro-democracy protesters left a trail of havoc and destruction across Hong Kong on Sunday, marching in defiance of a police ban and targeting government buildings and subway stations with vandalism.\nThe day’s violence, which began with masked demonstrators hurling petrol bombs and smashing windows at Admiralty and Wan Chai stations marked the 15th straight weekend that protests have gripped the city and raised questions over how the Asian financial hub’s future will be shaped.\nThe economic effects of the three-month crisis were brought into sharp focus on Sunday when Hong Kong International Airport released figures showing it had seen its biggest monthly drop in passengers for a decade in August as the protests prompted tourist and business visitors to cancel trips.\nFew see any signs of an imminent return to calm. “We are never going to back down before Beijing does,” said one 28-year-old protester who gave his name as Peter. “We have fought for 100 days now. What makes an

In [157]:
test_analysis_df.columns

Index(['Predicted', 'Test', 'Features'], dtype='object')